# Ejercicio: Crear tu propio chatbot para consultar archivos PDF

Objetivo:
Crear un chatbot que sea capaz de responder preguntas basadas en el contenido de archivos PDF ubicados en una carpeta. Implementar diferentes métodos de chunking para optimizar el proceso de consulta. Además, experimentar con la idea de trabajar en formato markdown. Por último, agregar la funcionalidad de chat_history para mantener una conversación fluida.

Instrucciones:
* Carga de archivos PDF: Trabajar con una carpeta de archivos que tenga al menos un archivo en formato PDF.
Utiliza herramientas como PyMuPDF, pdfplumber, o pypdf2 para extraer el texto de los archivos PDF.
* Procesamiento del texto con chunking: Explora diferentes estrategias de chunking para dividir el texto extraído en partes más manejables para el modelo:
 * Chunking basado en párrafos.
 * Chunking basado en número de tokens.
 * Chunking basado en líneas.
* Implementación del chatbot: Utiliza LangChain para crear el pipeline del chatbot. Usa el módulo DocumentLoader de LangChain para cargar los archivos PDF y procesarlos. Experimenta con los diferentes métodos de chunking para evaluar cuál da mejores resultados en términos de coherencia de las respuestas.
* Agregar chat_history: Una vez que tu chatbot esté funcionando correctamente, añade la funcionalidad de chat_history. Esto te permitirá mantener el contexto de la conversación para que el chatbot pueda referirse a preguntas anteriores en las respuestas actuales. Usa el módulo ConversationChain de LangChain para gestionar el historial de la conversación.



In [45]:
!pip install pymupdf
!pip install pymupdf pdfplumber PyPDF2 langchain openai gradio faiss-cpu tiktoken
!pip install -U langchain-community
!pip install -U langchain
!pip install DirectoryLoader
!pip install langchain-openai
!pip install pypdf
!pip install PyPDF2
!pip install --upgrade openai
!pip show langchain
!pip install openai
!pip show langchain-community
!pip install langchain langchain-community
!pip install langchain
!pip install gtts

ERROR: Could not find a version that satisfies the requirement DirectoryLoader (from versions: none)
ERROR: No matching distribution found for DirectoryLoader
Name: langchain
Version: 0.3.1
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community
Name: langchain-community
Version: 0.3.1
Summary: Community contributed LangChain integrations.
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, dataclasses-json, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [46]:
# Vamos a generar un archivo .py que contenga el código completo que necesitas

import openai
import PyPDF2  # Para la extracción de texto del PDF
import os
import gradio as gr
import requests
import time
from dotenv import load_dotenv

In [47]:
# Cargar variables de entorno para proteger claves API
load_dotenv()

False

In [48]:
# Configuración de la clave API desde la variable de entorno
api_key = os.getenv("OPENAI_API_KEY")  # Asegúrate de tener la clave en .env
endpoint = "https://aoai-ine.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-15-preview"

In [49]:
def send_request_to_model(prompt, user_input, api_key, endpoint, retries=5):
    """Envía la solicitud al modelo de OpenAI usando API REST con reintentos."""
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key,
    }

    payload = {
        "messages": [
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_input}
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800
    }

    attempt = 0
    while attempt < retries:
        try:
            response = requests.post(endpoint, headers=headers, json=payload)
            response.raise_for_status()
            response_json = response.json()
            # Eliminar el time.sleep aquí
            return response_json['choices'][0]['message']['content']
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                attempt += 1
                wait_time = 5 * (2 ** attempt)  # Retraso exponencial
                print(f"HTTP 429: Too Many Requests. Reintentando en {wait_time} segundos...")
                time.sleep(wait_time)
            else:
                raise SystemExit(f"Fallo en la solicitud. Error: {e}")
    raise SystemExit(f"Fallo en la solicitud después de {retries} reintentos.")


# --- Función para extraer texto de un PDF ---
def extract_text_from_pdf(pdf_path):
    """Extrae texto de un archivo PDF."""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text


In [50]:
# --- Estrategia de chunking por párrafos ---
def chunk_text_by_paragraphs(text, max_chunk_size=500):
    """Divide el texto en fragmentos (chunks) basados en párrafos."""
    paragraphs = text.split("\n\n")
    chunks = []
    current_chunk = ""

    for paragraph in paragraphs:
        if len(current_chunk) + len(paragraph) < max_chunk_size:
            current_chunk += paragraph + "\n\n"
        else:
            chunks.append(current_chunk)
            current_chunk = paragraph + "\n\n"

    if current_chunk:
        chunks.append(current_chunk)

    return chunks


In [51]:
def extract_text_from_pdf(pdf_path):
    """Extrae texto de un PDF usando PyPDF2."""
    text = ""
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"El archivo PDF no se encuentra en la ruta especificada: {pdf_path}")

    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text()
    except Exception as e:
        raise ValueError(f"Error al intentar leer el archivo PDF: {e}")

    if not text.strip():
        raise ValueError("No se pudo extraer texto del PDF o el archivo está vacío.")

    return text

In [52]:
# --- Cargar un único PDF y aplicar chunking ---
def load_single_pdf_and_chunk(pdf_path):
    """Carga un único PDF y lo divide en chunks."""
    text = extract_text_from_pdf(pdf_path)
    chunks = chunk_text_by_paragraphs(text)  # Puedes cambiar por chunking basado en tokens si lo prefieres
    return chunks

In [53]:
def chatbot_conversation(prompt, user_input, api_key, endpoint, chat_history=[]):
    """Maneja la conversación con el chatbot usando la API REST de OpenAI."""
    try:
        # Enviar la solicitud al modelo
        response = send_request_to_model(prompt, user_input, api_key, endpoint)

        # Actualizar el historial de chat
        chat_history.append(f"User: {user_input}")
        chat_history.append(f"Bot: {response}")

        return response, chat_history
    except Exception as e:
        print(f"Error en la conversación con el chatbot: {e}")
        return "", chat_history

In [54]:
# --- Cargar un único PDF y aplicar chunking ---
def load_single_pdf_and_chunk(pdf_path):
    """Carga un único PDF y lo divide en chunks."""
    text = extract_text_from_pdf(pdf_path)
    chunks = chunk_text_by_paragraphs(text)  # Puedes cambiar por chunking basado en tokens si lo prefieres
    return chunks

In [55]:
def chatbot_conversation(prompt, user_input, api_key, endpoint, chat_history=[]):
    """Maneja la conversación con el chatbot usando la API REST de OpenAI."""
    try:
        # Enviar la solicitud al modelo
        response = send_request_to_model(prompt, user_input, api_key, endpoint)

        # Actualizar el historial de chat
        chat_history.append(f"User: {user_input}")
        chat_history.append(f"Bot: {response}")

        return response, chat_history
    except Exception as e:
        print(f"Error en la conversación con el chatbot: {e}")
        return "", chat_history


In [66]:
# Interfaz de usuario con Gradio
def chatbot_ui(api_key, endpoint):
    """Crea la interfaz del chatbot usando Gradio."""
    try:
        # Especifica la ruta de tu archivo PDF directamente
        pdf_path = "/content/sample_data/proyectoManualSMR.pdf"

        # Verificar si el archivo existe
        if not os.path.exists(pdf_path):
            raise FileNotFoundError(f"Archivo PDF no encontrado en: {pdf_path}")

        # Cargar y chunkear el archivo PDF
        chunks = load_single_pdf_and_chunk(pdf_path)
        if not chunks:
            raise ValueError("El archivo PDF no contiene texto legible.")

        prompt = chunks[0]  # Usar el primer chunk como prompt inicial para el chatbot
        chat_history = []

        with gr.Blocks() as demo:
            # Insertar CSS para historial scrollable y con colores
            gr.HTML("""
            <style>
                #history_area {
                    height: 600px;
                    overflow-y: scroll;
                    border: 1px solid #ccc;
                    padding: 10px;
                    background-color: #f9f9f9;
                }
                .user { color: blue; }
                .bot { color: green; }
            </style>
            """)

            # Diseño con dos columnas: Una para la charla y otra para el historial
            with gr.Row():
                with gr.Column(scale=4):  # Columna principal con la charla
                    gr.Markdown("# Chatbot basado en un PDF")

                    # Campo de entrada para preguntas del usuario
                    user_input = gr.Textbox(label="Pregunta al chatbot", placeholder="Escribe tu pregunta aquí...")

                    # Botón para enviar mensaje
                    send_button = gr.Button("Enviar")

                    # Campo para mostrar la respuesta del chatbot
                    chat_output = gr.Textbox(label="Respuesta del chatbot", interactive=False)

                with gr.Column(scale=2):  # Columna del historial
                    # Título del historial del chat
                    gr.Markdown("## Historial del Chat")

                    # Campo para mostrar el historial con una barra de desplazamiento
                    history_output = gr.Markdown(elem_id="history_area")

                    # Botón para limpiar el historial, ubicado debajo del historial
                    clear_button = gr.Button("Limpiar historial")

            def interact(input_text):
                nonlocal chat_history, prompt

                # Generar respuesta y actualizar historial
                answer, updated_chat_history = chatbot_conversation(prompt, input_text, api_key, endpoint, chat_history)
                chat_history = updated_chat_history

                # Formatear el historial con colores
                formatted_history = ""
                for entry in chat_history:
                    if "User" in entry:
                        formatted_history += f"<p class='user'><strong>🧑 Usuario</strong>: {entry.split(':', 1)[1]}</p>"
                    elif "Bot" in entry:
                        formatted_history += f"<p class='bot'><strong>🤖 Bot</strong>: {entry.split(':', 1)[1]}</p>"

                # Borrar el input del usuario devolviendo una cadena vacía
                return answer, formatted_history, ""

            def clear_history():
                nonlocal chat_history
                chat_history = []
                return "", "", ""

            # Vincular las funciones a los eventos
            send_button.click(interact, inputs=[user_input], outputs=[chat_output, history_output, user_input])
            clear_button.click(clear_history, outputs=[chat_output, history_output, user_input])

        demo.launch(share=True, debug=True)

    except Exception as e:
        print(f"Error creando la interfaz del chatbot: {e}")



In [67]:
# Configuraciones de API ---
api_key = "033e4517fa324f76919184c2fed51809"  # Coloca tu API key correcta aquí
endpoint = "https://aoai-ine.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-15-preview"

 #Ejecutar la interfaz ---
if __name__ == "__main__":
    chatbot_ui(api_key, endpoint)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5ebdf1cd347504bcfe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://5ebdf1cd347504bcfe.gradio.live
